In [1]:
!pip install yt-dlp ffmpeg-python faster-whisper gradio transformers
!apt-get install ffmpeg -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


Importing required libraries

In [2]:
import os
import yt_dlp
import ffmpeg
import gradio as gr
from faster_whisper import WhisperModel
from transformers import pipeline

# Initialize summarizer pipeline from HuggingFace
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


Video Download + Audio Extraction Agent

In [4]:
def download_and_extract_audio(youtube_url):
    output_path = "downloads"
    os.makedirs(output_path, exist_ok=True)

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_path, '%(id)s.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=True)
        video_id = info_dict.get("id", None)
        filename = os.path.join(output_path, f"{video_id}.mp3")
        return filename


Transcriber Agent

In [5]:
def transcribe_audio(audio_path):
    model = WhisperModel("tiny", compute_type="int8")
    segments, _ = model.transcribe(audio_path)
    transcript = " ".join([seg.text for seg in segments])
    return transcript

Summarizer Agent

In [3]:
def summarize_text(transcript):
    chunks = [transcript[i:i+1000] for i in range(0, len(transcript), 1000)]
    summary = ""
    for chunk in chunks:
        result = summarizer(chunk, max_length=100, min_length=30, do_sample=False)
        summary += result[0]['summary_text'] + "\n"
    return summary.strip()

Note Generator Agent

In [6]:
def generate_notes(summary):
    notes = f"📝 **Study Notes**\n\n"
    for i, line in enumerate(summary.split('. ')):
        if line.strip():
            notes += f"{i+1}. {line.strip()}\n"
    return notes

GUI Interface

In [7]:
def ai_study_pipeline(youtube_url):
    try:
        audio_path = download_and_extract_audio(youtube_url)
        transcript = transcribe_audio(audio_path)
        summary = summarize_text(transcript)
        notes = generate_notes(summary)
        return transcript, summary, notes
    except Exception as e:
        return "Error occurred", "Error occurred", str(e)

gr.Interface(
    fn=ai_study_pipeline,
    inputs=gr.Textbox(label="Enter YouTube URL"),
    outputs=[
        gr.Textbox(label="Transcript"),
        gr.Textbox(label="Summary"),
        gr.Textbox(label="Study Notes")
    ],
    title="📚 AI Study Buddy",
    description="Multi-Agent System that helps you study YouTube lectures"
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd01f0c2871c65e91c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
